In [8]:
import tensorflow as tf
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from random import randint
import time
import matplotlib as mpl
# mpl.use('TkAgg')
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2 as cv
import plot_utils

from collections import defaultdict
from random import random, choice

import plot_utils
import mnist_data

from model import Model

IMAGE_SIZE = 28
RESULTS_DIR = 'results'
PRR_n_img_x = 10
PRR_n_img_y = 10
PRR_resize_factor = 1.0


dim_img = IMAGE_SIZE ** 2
learn_rate = 0.001
dim_z = 20
n_hidden = 500
batch_size = 128
add_noise = True

In [11]:
sess.close()

In [2]:
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [12]:
m = Model(20, keep_prob)
sess = tf.Session()
sess.run(tf.global_variables_initializer(), feed_dict={keep_prob : 0.9})

m.train(sess)
img = m.predict(sess)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
train_total_data [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
--- autoencoder ---


ValueError: Variable encoder/w0 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Users/josue/Devel/MCS/intelligent-systems/is-hw12/vae.py", line 10, in encoder
    w0 = tf.get_variable('w0', [x.get_shape()[1], n_hidden], initializer=w_init)
  File "/Users/josue/Devel/MCS/intelligent-systems/is-hw12/vae.py", line 70, in autoencoder
    mu, sigma = encoder(x_hat, n_hidden, dim_z, keep_prob)
  File "/Users/josue/Devel/MCS/intelligent-systems/is-hw12/model.py", line 62, in __init__
    self.x_hat, self.x, dim_img, dim_z, n_hidden, self.keep_prob)


In [9]:
image = cv.imread('results/predict.jpg')
plt.imshow(image)

In [7]:
img = m.predict(sess)
print(img)

y_PRR [[8.57958275e-06 4.18767013e-06 2.37597142e-05 ... 5.19520927e-06
  1.23045800e-06 1.83723705e-06]
 [5.93910045e-06 2.78733387e-05 9.14568227e-06 ... 4.25744474e-05
  3.37496886e-06 4.81609386e-05]
 [1.35786070e-06 1.43994419e-06 3.80270694e-06 ... 1.16027386e-05
  1.52932125e-06 2.76490641e-06]
 ...
 [2.87980311e-06 1.22979759e-06 1.35460095e-05 ... 1.25703427e-05
  1.13655403e-06 5.21970946e-07]
 [7.55478104e-05 6.58228892e-05 1.08679065e-04 ... 2.65824725e-04
  1.69351697e-05 1.16955571e-05]
 [1.73271983e-04 4.48794308e-05 9.57474113e-05 ... 3.39094840e-04
  8.26887772e-05 8.95919002e-05]]
None


In [10]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import random
from random import randint
import time

In [11]:
def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
    return result

In [41]:
from matplotlib import pyplot as plt

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2 as cv
import plot_utils

from collections import defaultdict
from random import random, choice

import plot_utils
import mnist_data
import vae

In [42]:
def sp_noise(image, prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape, np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

In [43]:
colors = [
    
]

def get_number(image, index):

    if (index > 99):
        row, col = 0, 0
    else:
        row = int(index / 10)
        col = index % 10
    rotation = randint(-5, 5)
    number = image[
        IMAGE_SIZE * row:IMAGE_SIZE * (row + 1), 
        IMAGE_SIZE * col:IMAGE_SIZE * (col + 1),
    ]
    number = cv.applyColorMap(number, cv.COLORMAP_JET)
    number = rotateImage(number, rotation)
    number = sp_noise(number, 0.03)
    return number

# sub = get_number(img, 25)
# res = cv.bitwise_not(sub)
# plt.imshow(res)

In [44]:
from random import choice
map_labels = defaultdict(list)

In [45]:
def get_label(arr):
    i = -1
    for i, x in enumerate(arr):
        if x == 1:
            break
    return i

In [46]:
train_total_data, train_size, _, _, test_data, test_labels = mnist_data.prepare_MNIST_data()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
train_total_data [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [47]:
for i in range(100):
    item = test_labels[i]
    map_labels[get_label(item)].append(i)

In [48]:
def get_captcha(input):
    input = str(input)
    img = cv.imread('results/predict.jpg', 0)
    captcha = []
    for s in input:
        i = int(s)
        x = choice(map_labels[i])
        sub = get_number(img, x)
        captcha.append(sub)
    return np.concatenate(captcha, axis=1)

In [49]:
sub = get_captcha(123213)
plt.imshow(sub)

In [50]:
img = cv.imread('results/predict.jpg', 0)

In [51]:
plt.imshow(img)

In [52]:
img

array([[0, 0, 0, ..., 0, 0, 2],
       [4, 3, 2, ..., 3, 0, 0],
       [3, 2, 0, ..., 2, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 0],
       [0, 2, 4, ..., 1, 0, 0]], dtype=uint8)